In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [42]:
openai = OpenAI()
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [ ]:
# A class to represent a Webpage
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
ed = Website("https://www.ansa.it/")
print(ed.title)
print(ed.text)

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += """The contents of this website is as follows; 
please provide a short summary of this website in markdown. 
If it includes news or announcements, then summarize these too.\n\n"""
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(ed))

In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
messages_for(ed)

In [ ]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [46]:
def summarize_locally(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model="llama3.2",
        messages=messages_for(website)
    )
    return response.choices[0].message.content

In [47]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    #summary = summarize(url)
    summary = summarize_locally(url)
    display(Markdown(summary))

In [48]:
display_summary("https://edwarddonner.com")

# Website Summary
### Tagline: Applying AI to Help People Discover Their Potential

The website is centered around Edward Donner, the founder and CTO of Nebula.io. The site outlines his passion for LLMs (Large Language Models) and the potential impact they can have.

**Key Content Highlights**

- **LLM Arena:** Engaging arena where LLMs compete against each other in battle of diplomacy and strategy.
- **About Author:** Edward Donner shares about his background as a founder, DJing, amateur music production enthusiast, Hacker News contributor, and industry expert in AI for talent management.

### Announcements
**Recent Updates**
- January 2025: LLM Workshop – Hands-on with Agents – resources available
- December 2024: Welcome to SuperDataScientists!
- November 13, 2024: Mastering AI and LLM Engineering – Resources published
- October 16, 2024: From Software Engineer to AI Data Scientist – Resources shared